In [1]:
import $ivy.`org.apache.spark::spark-sql:2.1.0` 
import $ivy.`InitialDLab:simba_2.11:1.0`
import $ivy.`org.slf4j:slf4j-jdk14:1.7.25`


import $ivy.$                                   

import $ivy.$                           

import $ivy.$                             


In [2]:
import org.slf4j.{Logger, LoggerFactory}
import org.apache.spark.sql.simba.{Dataset, SimbaSession}
import org.apache.spark.sql.simba.index.RTreeType
import org.apache.spark.sql.types.{StructType, StructField, LongType, DoubleType, IntegerType}

import org.apache.spark.sql.functions._
import scala.collection.JavaConverters._
import scala.collection.mutable.ListBuffer


import org.slf4j.{Logger, LoggerFactory}

import org.apache.spark.sql.simba.{Dataset, SimbaSession}

import org.apache.spark.sql.simba.index.RTreeType

import org.apache.spark.sql.types.{StructType, StructField, LongType, DoubleType, IntegerType}


import org.apache.spark.sql.functions._

import scala.collection.JavaConverters._

import scala.collection.mutable.ListBuffer


In [3]:
// spark://169.235.27.134:7077
val simba = SimbaSession.builder()
      .master("local[10]")
      .appName("MapMaximalDisks")
      .config("simba.index.partitions", 1024)
      .config("spark.cores.max", 32)
      .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/acald013/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/acald013/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-jdk14/1.7.25/slf4j-jdk14-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
18/05/10 18:02:38 INFO SparkContext: Running Spark version 2.1.0
18/05/10 18:02:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/05/10 18:02:39 INFO SecurityManager: Changing view acls to: acald013
18/05/10 18:02:39 INFO SecurityManager: Changing mod

simba: SimbaSession = org.apache.spark.sql.simba.SimbaSession@5f4c9561

In [4]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class ST_Point(id: Long, x: Double, y: Double, t: Int = -1)
case class Flock(start: Int, end: Int, ids: String, x: Double = 0.0, y: Double = 0.0)


defined class ST_Point
defined class Flock

In [5]:
import simba.implicits._
import simba.simbaImplicits._


import simba.implicits._

import simba.simbaImplicits._


In [6]:
val id = StructField("id", LongType)
val x = StructField("x", DoubleType)
val y = StructField("y", DoubleType)
val t = StructField("t", IntegerType)
val point_schema = StructType(Array(id, x, y, t))


id: StructField = StructField(id,LongType,true)
x: StructField = StructField(x,DoubleType,true)
y: StructField = StructField(y,DoubleType,true)
t: StructField = StructField(t,IntegerType,true)
point_schema: StructType = StructType(
  StructField(id,LongType,true),
  StructField(x,DoubleType,true),
  StructField(y,DoubleType,true),
  StructField(t,IntegerType,true)
)

In [7]:
val filename = "/home/acald013/Research/Datasets/Berlin/berlin0-2.tsv"

val pointset = simba.read
      .option("header", "false")
      .option("sep", "\t")
      .schema(point_schema)
      .csv(filename)
      .as[ST_Point]
      .cache()
val nPointset = pointset.count()


filename: String = "/home/acald013/Research/Datasets/Berlin/berlin0-2.tsv"
pointset: org.apache.spark.sql.Dataset[ST_Point] = [id: bigint, x: double ... 2 more fields]
nPointset: Long = 54732L

In [8]:
def getMaximalDisks(pointset: Dataset[ST_Point], t: Int, fraction: Double) = {
    pointset.filter(_.t == t).sample(false, fraction).map(d => Flock(t,t,s"${d.id}",d.x,d.y))
}

defined function getMaximalDisks

In [15]:
val maximalDisksCache = collection.mutable.Map[Int, Dataset[Flock]]()
var t = 0
maximalDisksCache.getOrElseUpdate(t, getMaximalDisks(pointset, t, 0.025))
t = 1
maximalDisksCache.getOrElseUpdate(t, getMaximalDisks(pointset, t, 0.050))


maximalDisksCache: collection.mutable.Map[Int, Dataset[Flock]] = Map(
  1 -> [start: int, end: int ... 3 more fields],
  0 -> [start: int, end: int ... 3 more fields]
)
t: Int = 1
res14_2: Dataset[Flock] = [start: int, end: int ... 3 more fields]
res14_4: Dataset[Flock] = [start: int, end: int ... 3 more fields]

In [16]:
t = 0
val A = maximalDisksCache.getOrElseUpdate(t, getMaximalDisks(pointset, t, 0.050))
t = 2
val B = maximalDisksCache.getOrElseUpdate(t, getMaximalDisks(pointset, t, 0.075))

A.count()
B.count()

A: Dataset[Flock] = [start: int, end: int ... 3 more fields]
B: Dataset[Flock] = [start: int, end: int ... 3 more fields]
res15_4: Long = 508L
res15_5: Long = 1328L

In [19]:
maximalDisksCache.apply(1).show(truncate = false)

+-----+---+-----+---------+---------+
|start|end|ids  |x        |y        |
+-----+---+-----+---------+---------+
|1    |1  |3808 |34570.076|23723.145|
|1    |1  |17395|19542.262|26040.434|
|1    |1  |14915|15320.889|14510.076|
|1    |1  |7477 |23533.218|22633.317|
|1    |1  |19056|21128.158|23570.095|
|1    |1  |10247|17599.327|16705.207|
|1    |1  |11699|21240.955|17517.009|
|1    |1  |10060|19269.585|16530.147|
|1    |1  |8325 |21550.966|20418.627|
|1    |1  |6205 |22997.345|21205.563|
|1    |1  |15119|16871.333|24260.321|
|1    |1  |5948 |24582.866|22205.247|
|1    |1  |14672|28662.935|12673.63 |
|1    |1  |1873 |32135.954|18351.013|
|1    |1  |16886|25886.479|21541.753|
|1    |1  |18971|31648.662|12679.064|
|1    |1  |15831|35610.404|12927.242|
|1    |1  |18302|17390.862|13923.548|
|1    |1  |8164 |20682.912|15859.364|
|1    |1  |3782 |23722.257|12412.957|
+-----+---+-----+---------+---------+
only showing top 20 rows



In [11]:
simba.close()